# Расширение списка оценочных слов с помощью word2vec
Идея - взять обученную модель word2vec, по очереди подавать в нее однословные элементы из seed, посмотреть на выдачу функции most_similar.
Очень вероятно, что слова, употребляющиеся в сходных контекстах с оценочными словами из списка seed тоже будут оценочными. 
Используются две модели. Одна с сайта RusVectores, обученная на википедии и НКРЯ, с большим словарём. Вторая - самостоятельно обученная на копусе твитов, собранном для sentiment-анализа. Мы предполагаем, что эмоционально окрашенные тексты будут содержать больше оценочной лексики и результат получится более точным и полным. 

In [1]:
from preprocessing import tokenize

In [2]:
import numpy as np
import pandas as pd
import regex as re
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors

In [3]:
with open('seed.txt') as file:
    seed_tokens = [tokenize(line, need_pos = True) for line in file.readlines()]

In [4]:
seed_tokens

[['аналог_NOUN', 'нет_PART'],
 ['подошва_NOUN'],
 ['вежливый_ADJ'],
 ['великолепный_ADJ'],
 ['вкусный_ADJ'],
 ['внимательный_ADJ'],
 ['большой_ADJ', 'выбор_NOUN'],
 ['вау_NOUN'],
 ['неплохой_ADJ'],
 ['идеальный_ADJ'],
 ['неприятный_ADJ'],
 ['понравиться_VERB'],
 ['рекомендовать_VERB'],
 ['ура_PART'],
 ['советовать_VERB'],
 ['отвратительный_ADJ'],
 ['отличный_ADJ'],
 ['положительный_ADJ'],
 ['отрицательный_ADJ'],
 ['приятный_ADJ'],
 ['напоминать_VERB', 'себя_PRON'],
 ['спасибо_NOUN'],
 ['радовать_VERB', 'глаз_NOUN'],
 ['совесть_NOUN'],
 ['молодец_NOUN'],
 ['прийти_VERB', 'еще_ADV', 'раз_NOUN'],
 ['орать_VERB'],
 ['улыбаться_VERB'],
 ['приветливый_ADJ'],
 ['отличный_ADJ'],
 ['замечательный_ADJ']]

In [5]:
# оставляем только одиночные оценочные слова, + убираем "подошву" и "орать", 
# потому что основное значение этих слов не оценочное и они дадут много мусора
good_seed_tokens = [
 ['вежливый_ADJ'],
 ['великолепный_ADJ'],
 ['вкусный_ADJ'],
 ['внимательный_ADJ'],
 ['вау_NOUN'],
 ['неплохой_ADJ'],
 ['идеальный_ADJ'],
 ['неприятный_ADJ'],
 ['понравиться_VERB'],
 ['рекомендовать_VERB'],
 ['ура_PART'],
 ['советовать_VERB'],
 ['отвратительный_ADJ'],
 ['отличный_ADJ'],
 ['положительный_ADJ'],
 ['отрицательный_ADJ'],
 ['приятный_ADJ'],
 ['спасибо_NOUN'],
 ['радовать_VERB'],
 ['молодец_NOUN'],
 ['улыбаться_VERB'],
 ['приветливый_ADJ'],
 ['отличный_ADJ'],
 ['замечательный_ADJ']]

## Готовая модель с rusvectores

In [13]:
# предобученная модель с сайта RusVectores
# 376 Мбайт, НКРЯ и  Википедия за декабрь 2017, размер копуса - 600 миллионов слов, размер словаря - 384 764,
# частотный порог - 40 слов
model = KeyedVectors.load_word2vec_format('ruwikiruscorpora_upos_skipgram_300_2_2018.vec.gz', binary=False)

In [14]:
all_similar_rv = []
for token_set in good_seed_tokens:
    if len(token_set) == 1:
        try:
            similar = model.most_similar(positive = token_set, negative = [], topn = 5)
            all_similar_rv.extend([i[0].split('_')[0] for i in similar])
        except:
            continue

In [15]:
all_similar_rv = set(all_similar_rv)
with open('w2v_rusvectores.txt', 'w') as file:
    file.write('\n'.join([i for i in all_similar_rv]))

In [16]:
len(all_similar_rv)

92

## Модель, обученная на корпусе твитов для sentiment-анализа 

In [30]:
model = KeyedVectors.load_word2vec_format('tweets.bin', binary=True)

In [42]:
all_similar_tweets = []
for token_set in good_seed_tokens:
    if len(token_set) == 1:
        token_set = [i.split('_')[0] for i in token_set]
        try:
            similar = model.most_similar(positive = token_set, negative = [], topn = 5)
            all_similar_tweets.extend([i[0] for i in similar])
        except:
            print(token_set[0]+' not in vocabulary')
            continue

In [47]:
all_similar_tweets = set(all_similar_tweets)
with open('w2v_tweets.txt', 'w') as file:
    file.write('\n'.join([i for i in all_similar_tweets]))

In [44]:
len(all_similar_tweets)

107

In [45]:
# Интересно, что из выдачи двух моделей совпали только 7 слов
len(all_similar_rv.intersection(all_similar_tweets))

7

In [46]:
with open('w2v_all.txt', 'w') as file:
    file.write('\n'.join([i for i in all_similar_rv.union(all_similar_tweets)]))

## Вывод
+ В списке, составленном с помощью готовой модели 92 слова, из них действительно оценочных - 72.
+ В списке, составленном с помощью готовой модели 107 слов, из них действительно оценочных - 63.
+ Из выдачи двух моделей совпали только 7 слов, значит использование второй модели здесь было оправданным и она достаточно обогатила список. 
+ В итоговом чистом списке из обеих моделей (w2v_clean_all.txt) 128 слов.  